<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.0 MB/s eta 0:00:00


In [3]:
import rasterio
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import glob
import os
import gdown
import zipfile
from skimage.transform import resize
from tensorflow.keras import layers
import random
from keras.utils import to_categorical
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
import cv2
from scipy.ndimage import generic_filter
from keras.models import load_model
from scipy.stats import mode
import glob
import os
import cv2
import numpy as np
import rasterio
from keras.utils import Sequence, to_categorical
import rasterio.plot
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
# Additional code can be added here if needed

In [4]:
os.mkdir('/content/image_chips')

In [5]:
# Get all .zip files
zip_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.zip')

# direct to the new folder
destination_path = '/content/image_chips/'

for zip_file in zip_files:
    # Get the base name of the zip file (without .zip extension)
    base_name = os.path.basename(zip_file)[:-4]

    # Create a new directory for this zip file
    unzip_dir = os.path.join(destination_path, base_name)
    os.makedirs(unzip_dir, exist_ok=True)

    # Unzip into the specific directory
    !unzip -q "$zip_file" -d "$unzip_dir"

warning [/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/Niihau_subset.zip]:  zipfile is empty
warning [/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/Lanai_subset.zip]:  zipfile is empty


In [6]:
import glob
import os
import pandas as pd

# Get all .csv files
csv_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.csv')


csv_list = []
for csv_file in csv_files:
    # Get the base name of the zip file (without .zip extension)
    base_name = os.path.basename(csv_file)[:-4]

    csv_loc = os.path.join(destination_path, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)


index_csv = pd.concat(csv_list, ignore_index=True)


In [7]:
print(index_csv.groupby('label')['percent'].sum())

print(index_csv.groupby('label')['percent'].count())

index_csv.groupby('label')['percent'].sum()/index_csv.groupby('label')['percent'].count()

label
13    339.165585
14     80.866539
15     26.158833
16      5.413750
17      3.640327
18      4.082653
Name: percent, dtype: float64
label
13    1431
14    1117
15    1285
16     199
17     205
18     170
Name: percent, dtype: int64


label
13    0.237013
14    0.072396
15    0.020357
16    0.027205
17    0.017758
18    0.024016
Name: percent, dtype: float64

In [ ]:
# '/content/image_chips/Maui10_subset/Images/10241_10753.tif'

In [8]:
index_csv

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name,subset
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,13,0.000919,8705_513.tif,Kauai10_subset
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,13,0.033161,9217_513.tif,Kauai10_subset
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,13,0.000130,13313_513.tif,Kauai10_subset
3,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,14,0.000137,13313_513.tif,Kauai10_subset
4,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,15,0.003502,13313_513.tif,Kauai10_subset
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,1537,1025,1025,513,26,703456.408765,704685.207789,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai_subset
4403,1537,1025,1025,513,26,703456.420411,704685.233874,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai11_subset
4404,7169,5121,6657,4609,221,716973.164648,718201.965448,2.303832e+06,2.305061e+06,15,0.001396,6657_4609.tif,Lanai11_subset
4405,7681,8705,7169,8193,383,718201.993245,719430.775154,2.295230e+06,2.296459e+06,15,0.001637,7169_8193.tif,Lanai11_subset


In [ ]:
index_csv.groupby(['tif_name', 'subset']).size().reset_index(name='Count')

,tif_name,subset,Count
0,10241_10241.tif,Oahu11_subset,1
1,10241_1025.tif,Kauai10_subset,2
2,10241_1025.tif,Kauai_subset,2
3,10241_10753.tif,Maui10_subset,2
4,10241_10753.tif,Maui_subset,2
...,...,...,...
2350,9729_7681.tif,Oahu_subset,1
2351,9729_8193.tif,Kauai10_subset,1
2352,9729_8193.tif,Kauai_subset,1
2353,9729_9729.tif,Oahu11_subset,2


In [42]:
import pandas as pd

# Aggregate the 'percent' values for duplicate 'tif_name' entries
agg_df = index_csv.groupby(['tif_name', 'label']).agg({'percent': 'mean'}).reset_index()

# Pivot the data to create columns for each unique 'label'
pivot_df = agg_df.pivot(index='tif_name', columns='label', values='percent').reset_index()

# Fill NaN values with 0 if needed
pivot_df = pivot_df.fillna(0)

# Merge with the original dataset to keep the other relevant information
summary_df = index_csv[['tif_name', 'xmin', 'xmax', 'ymin', 'ymax', 'subset']].drop_duplicates().merge(pivot_df, on='tif_name')

# Print the resulting DataFrame
print(summary_df)

             tif_name           xmin           xmax          ymin  \
0        8705_513.tif  438833.983073  440062.758637  2.457574e+06   
1        8705_513.tif  438834.042812  440062.804904  2.457574e+06   
2        9217_513.tif  440062.834396  441291.605845  2.457574e+06   
3        9217_513.tif  440062.792526  441291.578095  2.457574e+06   
4       13313_513.tif  449893.209832  451122.010258  2.457574e+06   
...               ...            ...            ...           ...   
2350  10241_10241.tif  598860.774759  600089.564487  2.375564e+06   
2351     1025_513.tif  703456.408765  704685.207789  2.313662e+06   
2352     1025_513.tif  703456.420411  704685.233874  2.313662e+06   
2353    7169_8193.tif  718201.993245  719430.775154  2.295230e+06   
2354    7681_8705.tif  719430.806294  720659.606473  2.294002e+06   

              ymax          subset        13        14        15   16   17  \
0     2.458802e+06  Kauai10_subset  0.000965  0.000000  0.000000  0.0  0.0   
1     2.458802e

In [43]:
summary_df

,tif_name,xmin,xmax,ymin,ymax,subset,13,14,15,16,17,18
0,8705_513.tif,438833.983073,440062.758637,2.457574e+06,2.458802e+06,Kauai10_subset,0.000965,0.000000,0.000000,0.0,0.0,0.0
1,8705_513.tif,438834.042812,440062.804904,2.457574e+06,2.458802e+06,Kauai_subset,0.000965,0.000000,0.000000,0.0,0.0,0.0
2,9217_513.tif,440062.834396,441291.605845,2.457574e+06,2.458802e+06,Kauai10_subset,0.033218,0.000000,0.000000,0.0,0.0,0.0
3,9217_513.tif,440062.792526,441291.578095,2.457574e+06,2.458802e+06,Kauai_subset,0.033218,0.000000,0.000000,0.0,0.0,0.0
4,13313_513.tif,449893.209832,451122.010258,2.457574e+06,2.458802e+06,Kauai10_subset,0.000863,0.009518,0.096784,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2350,10241_10241.tif,598860.774759,600089.564487,2.375564e+06,2.376793e+06,Oahu11_subset,0.000000,0.000000,0.002010,0.0,0.0,0.0
2351,1025_513.tif,703456.408765,704685.207789,2.313662e+06,2.314891e+06,Lanai_subset,0.000000,0.000000,0.000828,0.0,0.0,0.0
2352,1025_513.tif,703456.420411,704685.233874,2.313662e+06,2.314891e+06,Lanai11_subset,0.000000,0.000000,0.000828,0.0,0.0,0.0
2353,7169_8193.tif,718201.993245,719430.775154,2.295230e+06,2.296459e+06,Lanai11_subset,0.000000,0.000000,0.001637,0.0,0.0,0.0


In [32]:
dummy_vec = [0] * 5 + [1] * 3

print(dummy_vec)

np.random.shuffle(dummy_vec)

print(dummy_vec)

[0, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 0, 1, 1, 0, 0, 0]


In [54]:
# Calculate the number of rows for training (80%) and validation (20%)
total_rows = len(summary_df)
train_rows = int(0.9 * total_rows)
val_rows = total_rows - train_rows

# Generate an array of 0s (80% of total) and 1s (20% of total) randomly
random_assignment = np.array([0] * train_rows + [1] * val_rows)

# Shuffle the array to randomize the assignment
np.random.shuffle(random_assignment)

# Add the random assignment as 'split' column to your DataFrame
summary_df['random_split'] = random_assignment

# # Save the updated DataFrame to a CSV file
# index_csv.to_csv('index_random.csv', index=False)
summary_df.columns = summary_df.columns.astype(str)
print("Random split column added to the DataFrame.")


Random split column added to the DataFrame.


In [55]:
import pandas as pd

# Aggregate the 'percent' values for duplicate 'tif_name' entries
agg_df = summary_df.groupby(['random_split']).agg({'13': 'mean','14':'mean','15': 'mean','16':'mean','17': 'mean','18':'mean'}).reset_index()

print(agg_df)

   random_split        13        14        15        16        17       18
0             0  0.164668  0.035724  0.012214  0.003971  0.003503  0.00381
1             1  0.174207  0.034783  0.013664  0.002926  0.001522  0.00337
